# 32 스케쥴러

## 1. 스케쥴러

(1) 스케쥴러란
- 특정시간이나 정해진 간격에 따라 어떤 함수가 자도으로 실행되는 처리
  - ex. 1마다 한 번씩 자동으로 수행되는 기능, 매일 자정에 자동으로 수행되는 기능
    - apscheduler: 파이썬 공식 문서에서 소개됨.
      - pip install --upgrade apscheduler
        - 쓰는 방법잘모를땐 ?를 함수 앞에 붙여보자

(2) 작업준비
- 패키지 참조
  
- 스케쥴에 따라 자동으로 실행될 기능 정의


In [1]:
# 작업 준비
## 패키지 참조             
from apscheduler. schedulers. background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger
from apscheduler.jobstores.base import JobLookupError
import datetime as dt
import time

## 스케쥴에 따라 자동으로 실행될 기능
def myjob(name):
    currentTime = dt.datetime.now()
    timeFormat = currentTime.strftime("%Y/%m/%d %H:%M:%S")
    print("[%s] I'm working... | %s" % (name, timeFormat))

myjob("shin")

[shin] I'm working... | 2023/12/01 14:45:36


### (3) 스케쥴러 등록
- 1) 정해진 간격마다 실행하기
  
  - 스케쥴러 객체 생성
  - 스케쥴 작업 등록
  - 스케쥴러에서 작업 제거

- 2) cron 표현식으로 설정하기
  - cron 표현식
    - Linux, Mac 등에서 작업 스케쥴러를 등록할 때 사용하는 시간 단위 설정 표현식
    - 공백으로 구분하는 7자리의 값으로 구성
  - 값의 설정 방법

In [2]:
# 스케쥴러 등록
# 1) 정해진 간격마다 실행하기
    # 스케쥴러 객체 생성
sched = BackgroundScheduler()
sched.start()
    # 스케쥴 작업 등록
sched.add_job(myjob, "interval", seconds = 3, args = ["kim"], id = 'myjob1')

    # 스케쥴러에서 작업 제거
sched.remove_job("myjob1")
sched.shutdown()

# 2) cron 표현식으로 설정
sched = BackgroundScheduler()
sched.start()
# 매 분 2 초
sched.add_job(myjob, 'cron', second = 2, args = ["kim"], id = 'myjob2')
# 매초(*)마다 2초 간격으로
sched.add_job(myjob, 'cron', second='*/2', args = ["hong"], id = 'myjob3')



<Job (id=myjob3 name=myjob)>

[hong] I'm working... | 2023/12/01 14:45:50
[hong] I'm working... | 2023/12/01 14:45:52


In [3]:
# 스케쥴러에서 작업 제거

# 작업 제거
    # 1회 수행 후 영구 종료된 작업은 제거할 수 없음. 
sched.remove_job('myjob2')
sched.remove_job('myjob3')
# 스케쥴러 멈춤
sched.shutdown()

### 2) cron 표현식으로 석정하기

#### cron 표현식
Linux, Mac 등에서 작업 스케쥴러를 등록할 때 사용하는 시간 단위 설정 표현식

공백으로 구분하는 7자리의 값으로 구성됨

```shell
*******
```

각 자리는 순서대로 '<초>,<분>,<시>,<일>,<월>,<요일>,<년>'을 의미함

#### 값의 설정 방법

| 필드 | 허용되는 값 | 허용되는 특수문자|
|---|---|---|
|초 (seconds)|0~59|`,` `-` `*` `/`|
|분 (minutes)|0~59|`,` `-` `*` `/`|
|시 (hours)|0~23|`,` `-` `*` `/`|
|일 (Day)|1~30|`,` `-` `*` `/`|
|월 (Month)|0~12|`,` `-` `*` `/` `L` `#`|
|요일 (Day of weeks)|0~12|`,` `-` `*` `/` `L` `#`|
|년 (Year)|1970~2099|`,` `-` `*` `/`|


참조 : https://crontab.cronhub.io/ 

In [4]:
# 3) 특정 시각에 수행

sched = BackgroundScheduler()
sched.start()
targetDate = dt.datetime(2023, 12, 1, 11, 43, 0)
sched.add_job(myjob, 'date', run_date=targetDate, args=["park"], id='myjob4')

# 예약된 시각에 1회 수행하고 종료하므로 remove_job을 처리할 필요가 없다.
#sched.remove_job("myjob4")
sched.shutdown()

In [5]:
# 03. 메일링 리스트 개선 (스케쥴러에 두기)


import MyMailer
import concurrent.futures as futures

today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "운영지원팀 <jcdasom@gmail.com>"
subjecTempl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mail/content.txt', 'r', encoding = 'utf-8') as f:
    contentTmpl = f.read()


In [6]:
## 수행할 작업은 def 로 함수처리 해둔다.

def sendmail():
    startTime = dt.datetime.now()
    
    with open("mail/mail_list.csv", "r", encoding = "euc-kr") as f:
        csv = f.readlines()

        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            for line in csv:
                name, email, file1, file2 = line.strip().split(",")
                toAddr = "{name} <{email}>".format (name = name, yy=year, mm=month)
                subject = subjectTmpl.format(name=name, yy=year, mm=month)
                content = contentTmpl.format(name=name, yy=year, mm = month, dd=day)

                executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

    endTime = dt.datetime.now()            
    workTime = endTime - startTime
    print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

    sched = BackgroundScheduler()
    sched.start()
    sched.add_job(sendmail, 'cron', second = '*/5', id = 'mymailer')




In [7]:
sched = BackgroundScheduler()
sched.start()

sched.add_job(sendmail, 'cron', second = '*/5', id = 'mymailer')

<Job (id=mymailer name=sendmail)>

Job "sendmail (trigger: cron[second='*/5'], next run at: 2023-12-01 14:46:35 KST)" raised an exception
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_63180\4208496587.py", line 12, in sendmail
    toAddr = "{name} <{email}>".format (name = name, yy=year, mm=month)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyError: 'email'
Job "sendmail (trigger: cron[second='*/5'], next run at: 2023-12-01 14:46:40 KST)" raised an exception
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [8]:
sched.remove_job("mymailer")
sched.shutdown()